# Validalab, s'informer en confiance

Dans ce **TP**, nous allons exploiter la base de données de **Validalab**. <br>
Une organisation à but non lucratif donc le but est d'aider les citoyens à mieux s'informer et à être acteurs de leur consommation d'informations.<br>
Elle a été initiée par **Jean-Marc Guerin**

![jmg.png](../data/images/jmg.png)

Plutôt que de faire du fact-checking, **Validalab** prend le parti d'informer les utilisateurs sur la source d'information. <br>
Ainsi, sur cette [application](http://app.validalab.fr/), on peut retrouver différentes informations agrégées sur les médias français.<br>
>Les données ont été scrappées sur différents dites et ingérés dans une base de données **Neo4j**. Le [dictionnaire des données](https://docs.google.com/spreadsheets/d/17iylS3y-xRVZLFMOuyz-o5Oy_MSlO_jvQoY8LBmH1LQ/edit#gid=1217348665) décrit les **types de données**, les **propriétés** et les **relations** du graphe de Validalab.

## Installation du Driver python de Neo4j 

In [38]:
!pip install neo4j

## Importation des packages 

In [39]:
from neo4j import GraphDatabase, basic_auth

## Informations de connexion à la base de données

In [43]:
password = "dfg"
ip = "163.172.110.238"
driver = GraphDatabase.driver('bolt://'+ip,auth=basic_auth("neo4j", password))

In [44]:
db = driver.session(database='jmdemo')

### 1- Introduction
Pour exécuter une requête **CYPHER** via le driver python de Neo4j, on utilise la méthode `run` de l'objet `Session`, soit `db` dans notre cas.

Exemple : Lister les différents types de données

In [48]:
# Lister les différents type de données
results = db.run("""
MATCH (n) RETURN DISTINct labels(n)
""")
results.data()

[{'labels(n)': ['Website']},
 {'labels(n)': ['Pinterest']},
 {'labels(n)': ['Facebook']},
 {'labels(n)': ['Twitter']},
 {'labels(n)': ['Linkedin']},
 {'labels(n)': ['Entity']},
 {'labels(n)': ['Youtube']},
 {'labels(n)': ['Wikipedia']},
 {'labels(n)': ['Instagram']}]

#### Help
Consultons l'aide de la méthode run

In [ ]:
help(db.run)

Elle a 2 principaux arguments(`query` et `parameters`) et des arguments indéfinis `**kwargs`.

- `query`: comme l'indique la docstring, c'est tout simplement la requête CYPHER
- `parameters`: il s'agit d'un dictionnaire de paramètres utilisables dans la requête précédente

In [ ]:
# Afficher les informations sur l'entité pertant le nom "Le Monde SA"
results = db.run("""
    MATCH (n:Entity {name: $entity_name})
    RETURN n
    """, {"entity_name": "Le Monde SA"})
results.data()

### 2- Clause MATCH

Combien de nœuds de type **Entity**, **Website**, **Wikipedia** y a-t-il dans la base de données ?
> Créer une fonction permettant de compter le nombre de noeuds pour un Label donné.

**Lien utile :** https://neo4j.com/docs/cypher-manual/current/clauses/match/#basic-node-finding

In [ ]:
def count_nodes(entity_type):
    results = db.run(f"""
    MATCH (n:{entity_type}) RETURN count(*) as count
    """)
    return results.data()[0].get('count')

In [ ]:
for label in ["Entity", "Website", "Wikipedia"]:
    print(f"There is {count_nodes(label)} nodes with the type {label}")

---
Lister les 10 premiers nœuds de type **Entity**

In [ ]:
def find_nodes(entity_type,limit=10):
    results = db.run(f"""
    MATCH (n:{entity_type}) RETURN n LIMIT {limit}
    """)
    return results.data()

In [ ]:
find_nodes("Entity", limit=1)

---
Afficher les nœuds de type Entity sous forme de **DataFrame**<br>
**N.B:** la méthode `to_df()` peut aider.

In [ ]:
results = db.run("""
MATCH (n:Entity) RETURN n.Diplo_mediaPeriodicite, n.name, n.Diplo_typeLibelle LIMIT 10
""")
results.to_df()

### 3- Clause WHERE

Créer une fonction qui permet de retrouver des sites contenant une chaine de caractères.<br>
**Exemple :** lemonde, valeurs, etc...
<br>
**Lien utile :** https://neo4j.com/docs/cypher-manual/current/clauses/where/

In [ ]:
def find_website(string):
    results = db.run(f"""
    MATCH (w:Website)
    WHERE w.name =~"(?i).*{string}.*" 
    RETURN w
    """)
    return results.data()

In [ ]:
find_website('lemonde')

### 4- RELATIONS

---
Créer une fonction qui retourne le résumé **Wikipedia** pour un site web donné.
> Rappel: Dans la base de données, nous avons les données Wikipedia. Le résumé des noeuds Wikipedia correspond à la propriété ``summary`.

In [ ]:
def get_summary(site_name):
    results = db.run("""
    MATCH (n {name: $site_name})-[:OWNED_BY]->(e:Entity)<-[:OWNED_BY]-(w:Wikipedia)
    RETURN w.summary
    """, {"site_name":site_name})
    return results.data()[0].get('w.summary')

In [ ]:
get_summary('lemessager.fr')

---
Le Gorafi, Le Monde et Valeurs Actuelles sont-ils des journaux fiables ?
Pour répondre à cette question, on peut lister les **citations** de ces médias.<br>
> Les `citations` d'un site, dans notre contexte,  sont les recommandations(<span style="background:green; color:white">positives</span> ou <span style="background:red; color:white">négative</span>) de ce site par des entités quelconques.

In [ ]:
def list_recommendations(site_name):
    results = db.run("""
    MATCH(w:Website{name:$site_name})<-[reco:RECOMMENDS]-(r) 
    RETURN r.name as recommender, reco.weight as weight, 
    reco.meaning as meaning, reco.sourceURL as sourceURL
    """, {"site_name":site_name})
    return results.to_df()

In [ ]:
list_recommendations("lemonde.fr")

In [ ]:
list_recommendations("legorafi.fr")

In [ ]:
list_recommendations("valeursactuelles.com")

### 5- Investigations

---
Créer une fonction pour déterminer les propriétaires finaux d'un site web.
> Les propriétairs finaux sont ceux au sommet de la chaine. C'est à dire, ceux qui n'ont personne qui les possède.

In [ ]:
def final_onwers(site_name):
    results = db.run("MATCH (w:Website) "
                "WHERE w.name =~ $site_name "
                "MATCH (w)-[:OWNED_BY*]->(e:Entity)"
                "MATCH (e)<-[:OWNED_BY*]-(n) "
                "WITH e, n "
                "WHERE NOT (e)-[:OWNED_BY]->()"
                "RETURN DISTINCT(e.name) as proprietaire",
                {"site_name": f"(?i).*{site_name}.*"})
    return [proprio.get('proprietaire') for proprio in results.data()]

In [ ]:
final_onwers('lemessager.fr')

---
Créer une fonction pour déterminer le nombre de médias que possèdent chacun des propriétaires finaux trouvés.

In [ ]:
def medias_by_owners(site_name):
    results = db.run("MATCH (w:Website) "
                "WHERE w.name =~ $site_name "
                "MATCH (w)-[:OWNED_BY*]->(e:Entity)"
                "MATCH (e)<-[:OWNED_BY*]-(n) "
                "WITH e, n "
                "WHERE NOT (e)-[:OWNED_BY]->()"
                "RETURN DISTINCT e.name as proprietaire, count(n) as nb_medias",
                {"site_name": f"(?i).*{site_name}.*"})
    return results.to_df().sort_values(by="nb_medias", ascending=False)

In [ ]:
medias_by_owners('lemonde.fr')